# Imports

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

!pip install transformers
!pip install sentencepiece
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!pip install nlpaug
!pip install ray


     |████████████████████████████████| 248 kB 8.5 MB/s 
     |████████████████████████████████| 9.9 MB 62.9 MB/s 
     |████████████████████████████████| 6.5 MB 51.2 MB/s 
     |████████████████████████████████| 311 kB 58.3 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 3.8 MB 53.0 MB/s 
     |████████████████████████████████| 1.7 MB 50.7 MB/s 
     |████████████████████████████████| 1.2 MB 54.5 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 61.9 MB/s 
     |████████████████████████████████| 181 kB 67.5 MB/s 
     |████████████████████████████████| 144 kB 71.8 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 212 kB 72.3 MB/s 
     |████████████████████████████████| 134 kB 70.4 MB/s 
     |████████████████████████████████| 1.1 MB 66.4 MB/s 
     |███████████████

     |████████████████████████████████| 125 kB 8.0 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
     |████████████████████████████████| 410 kB 7.9 MB/s 
     |████████████████████████████████| 59.6 MB 1.2 MB/s 
     |████████████████████████████████| 175 kB 72.3 MB/s 


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import tensorflow as tf


import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel

from transformers import RobertaTokenizer
from transformers import RobertaPreTrainedModel, RobertaModel

import nlpaug.augmenter.word as naw
from sklearn.utils import shuffle
from ray import tune

import pandas as pd
import numpy as np
import os

import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
rng_seed = 90
torch.manual_seed(rng_seed)

#Data Loading and Examination

## Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer


# Class from 'dont_patronize_me.py' to merge labels into binary classfications

class DontPatronizeMe:

  def __init__(self, train_path, test_path):

    self.train_path = train_path
    self.test_path = test_path
    self.train_task1_df = None
    self.train_task2_df = None
    self.test_set_df = None

  def load_task1(self):
    """
    Load task 1 training set and convert the tags into binary labels. 
    Paragraphs with original labels of 0 or 1 are considered to be negative examples of PCL and will have the label 0 = negative.
    Paragraphs with original labels of 2, 3 or 4 are considered to be positive examples of PCL and will have the label 1 = positive.
    It returns a pandas dataframe with paragraphs and labels.
    """
    rows=[]
    with open(self.train_path) as f: #no joining, just the exact path otherwise, it seems to fail
      for line in f.readlines()[4:]:   #[4:]

        par_id=line.strip().split('\t')[0]
        art_id = line.strip().split('\t')[1]
        keyword=line.strip().split('\t')[2]
        country=line.strip().split('\t')[3]
        t=line.strip().split('\t')[4]#.lower()
        l=line.strip().split('\t')[-1]
        # if l=='3':
        #   print(t)
        #   print("----------")
        if l=='0' or l=='1':
          lbin=0
        else:
          lbin=1
        rows.append(
          {'par_id':par_id,
          'art_id':art_id,
          'keyword':keyword,
          'country':country,
          'text':t, 
          'label':lbin, 
          'orig_label':l
          }
          )
    df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text', 'label', 'orig_label']) 
    self.train_task1_df = df

 

In [ ]:
#this is where I hardcoded it in
dpm = DontPatronizeMe('/content/drive/MyDrive/Colab Notebooks/NLP CW/dontpatronizeme_pcl.tsv', None) 
dpm.load_task1()

open('/content/drive/My Drive/Colab Notebooks/NLP CW/train_semeval_parids-labels.csv')
open('/content/drive/My Drive/Colab Notebooks/NLP CW/dev_semeval_parids-labels.csv')


<_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/NLP CW/dev_semeval_parids-labels.csv' mode='r' encoding='UTF-8'>

In [ ]:
trids = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP CW/train_semeval_parids-labels.csv')
teids = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP CW/dev_semeval_parids-labels.csv')

trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

## Formatting


In [ ]:
# Rebuilt Official Train Set without keyword and country codes

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      #'par_id':parid,
      'texts':text,
      'labels':label
  })

In [ ]:
trdf1 = pd.DataFrame(rows)

In [ ]:
# Rebuilt Official train set with keyword and country codes

rows_additional = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve columns with training information
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  rows_additional.append({
      #'par_id':parid,
      'texts':text,
      'labels':label,
      'country':country,
      'keyword':keyword
  })

In [ ]:
extra_cols_train_set = pd.DataFrame(rows_additional)

In [ ]:
# Rebuild Official Test Set

rows_test = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows_test.append({
      #'par_id':parid,
      'texts':text,
      'labels':label,
  })

In [ ]:
tedf1 = pd.DataFrame(rows_test)

## Split Official Training into Training and Validation

In [ ]:
# Split data into training and validation
trainset, valset = train_test_split(trdf1, test_size=0.2, random_state = 10)

trainset.index = pd.RangeIndex(len(trainset.index))
valset.index = pd.RangeIndex(len(valset.index))

trainset.index = range(len(trainset.index))
valset.index = range(len(valset.index))


# Training Dataset Creation

## Downsample not condescending

In [ ]:
# Downsample negative instances
pcldf = trainset[trainset.labels==1]
npos = len(pcldf)

trainset_downsample = pd.concat([pcldf,trainset[trainset.labels==0][:npos*2]])
trainset_downsample = shuffle(trainset_downsample.reset_index(drop=True))

In [ ]:
trainset.labels.value_counts()

In [ ]:
trainset_downsample.labels.value_counts()

In [ ]:
trainset_downsample.head()

## Augmentation (via Contextual Embeddings)

In [ ]:
# Upload saved augmented dataset to save time if desired
#trainset_downsample_aug = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP CW/augmentations_150.csv')

In [ ]:
# Adapted from https://neptune.ai/blog/data-augmentation-nlp
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-cased', action="substitute") 

In [ ]:
# Function to augment texts from dataframe and output a dataframe with augmentations

def augment_text(df,samples_1=150, samples_0=150, pr=0.2):
    aug.aug_p=pr
    new_text_1=[]
    new_text_0=[]
    
    # select minority class samples
    df_1 = df[df.labels==1].reset_index(drop=True)

    # data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_1),samples_0)):

            text = df_1.iloc[i]['texts']
            augmented_text = aug.augment(text)
            new_text_1.append(augmented_text)
    
    # select majority class samples
    df_0 = df[df.labels==0].reset_index(drop=True)

    # data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_0),samples_1)):

            text = df_0.iloc[i]['texts']
            augmented_text = aug.augment(text)
            new_text_0.append(augmented_text)
    
    # dataframe
    new_1=pd.DataFrame({'texts':new_text_1,'labels':1})
    new_0=pd.DataFrame({'texts':new_text_0,'labels':0})
    df=shuffle(df.append(new_1).reset_index(drop=True))
    df=shuffle(df.append(new_0).reset_index(drop=True))
    return df


In [ ]:
# Apply synonym replacement augmentation to downsampled trainset
augmentations = trainset_downsample.copy()
trainset_downsample_aug = augment_text(augmentations)

In [ ]:
# Save augmented dataset to a csv
trainset_downsample_aug.to_csv('/content/drive/My Drive/Colab Notebooks/NLP CW/augmentations_150.csv') 

## Custom Dataset Class Creation

In [ ]:
# Define DpmDataset class which allows us to control how we handle the iteration and batches
# From lab 5
# At each iteration over the dataset object, the function ''__get_item__'' is called and returns a list of dictionnaries with the paragraphs and their labels
# Then, the ''collate_fn'' function will process the list of samples into their encodings and return a batch when called by the iterator during training

class DpmDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set):

        self.tokenizer = tokenizer
        self.texts = input_set['texts']
        self.labels = input_set['labels']
        
    def collate_fn(self, batch):

        texts = []
        labels = []

        for b in batch:
            texts.append(b['text'])
            labels.append(b['label'])

        #The maximum sequence size for BERT is 512 but here the tokenizer truncate sentences longer than 128 tokens.  
        # We also pad shorter sentences to a length of 128 tokens
        encodings = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=300)
        labels_all = {}
        encodings['label'] =  torch.tensor(labels)
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if idx < len(self.texts):
          item = {'text': self.texts[idx],
                'label': self.labels[idx]}
          return item
        else:
          return

In [ ]:
# Use a pre-made tokenizer that was used for the BERT model 
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# Creat datasets for training and evaluation using Dpm class
train_dataset = DpmDataset(tokenizer, trainset_downsample_aug) # with downsampling and augmentation
val_dataset = DpmDataset(tokenizer, valset)

# Define BERT Model

In [ ]:
# Define our model on top of BERT for classification

class BERT_dpm(BertPreTrainedModel):
#class BERT_dpm(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # BERT Model
        self.bert = BertModel(config)
        #self.bert = RobertaModel(config)

        self.projection = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                                torch.nn.Linear(config.hidden_size, 2))

        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Logits 
        logits = self.projection(outputs[1])
        return logits

# Training

## Define Training Function

In [ ]:
# Trainer class to incorporate a compute_loss fn we want

class Trainer_dpm(Trainer):
    def compute_loss(self, model, inputs):

        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs)
        
        labels = labels['label']

        loss_task = nn.CrossEntropyLoss()            
        loss = loss_task(outputs.view(-1, 2), labels.view(-1))

        loss = loss
        
        return loss

NameError: ignored

In [ ]:
# Define arguments for Train_dpm and initiate training

def main_dpm(learning_rate, per_device_train_batch_size, num_train_epochs):

    #call our custom BERT model and pass as parameter the name of an available pretrained model
    model = BERT_dpm.from_pretrained("bert-base-cased")
    
    training_args = TrainingArguments(
        output_dir='./experiment/dpm',
        learning_rate = learning_rate, 
        lr_scheduler_type = 'linear',
        logging_steps = 50,
        per_device_train_batch_size = per_device_train_batch_size, 
        num_train_epochs = num_train_epochs,  
    )
    trainer = Trainer_dpm(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/dpm_bert_finetuned/')

## Run Training

In [ ]:
# Define hyperparameters
learning_rate = 0.00001
per_device_train_batch_size = 8
num_train_epochs = 3

# Train 
main_dpm(learning_rate, per_device_train_batch_size, num_train_epochs)

# Evaluation

## Evaluation Functions

In [ ]:
# Function to extract predicted label

def predict_dpm(input, tokenizer, model): 
  model.eval()
  encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=300)
  
  output = model(**encodings)
  preds = torch.max(output, 1)


  return {'prediction':preds[1], 'confidence':preds[0]} # index 0 is values, index 1 is indices

In [ ]:
# Function to evaluate model on validation set

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader): 

      labels = {}
      labels['label'] = data['label']

      paras = data['text']

      pred = predict_dpm(paras, tokenizer, model)

      #preds.append(pred)
      preds.append(pred['prediction'].tolist())
      tot_labels.append(labels['label'].tolist())

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score
  report = classification_report(tot_labels, preds, target_names=["negative","positive"], output_dict= True)

  return report

## Evaluate Validation Set

In [ ]:
# Evaluate

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

model_name = './models/dpm_bert_finetuned/' 
model = BERT_dpm.from_pretrained(model_name)

val_loader = DataLoader(val_dataset)

report = evaluate(model, tokenizer, val_loader)

print(report)

print(report['positive']['f1-score'])

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
lo

  0%|          | 0/1675 [00:00<?, ?it/s]

{'negative': {'precision': 0.9702467343976778, 'recall': 0.8883720930232558, 'f1-score': 0.9275060700659036, 'support': 1505}, 'positive': {'precision': 0.43434343434343436, 'recall': 0.7588235294117647, 'f1-score': 0.5524625267665954, 'support': 170}, 'accuracy': 0.875223880597015, 'macro avg': {'precision': 0.7022950843705561, 'recall': 0.8235978112175102, 'f1-score': 0.7399842984162495, 'support': 1675}, 'weighted avg': {'precision': 0.9158565487205307, 'recall': 0.875223880597015, 'f1-score': 0.8894419492534364, 'support': 1675}}
0.5524625267665954


# Evaluate Test Set

In [ ]:
# Evaluate

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

model_name = './models/dpm_bert_finetuned/' 
model = BERT_dpm.from_pretrained(model_name)

test_dataset = DpmDataset(tokenizer, tedf1)
test_loader = DataLoader(test_dataset)

report = evaluate(model, tokenizer, test_loader)

print(report)

print(report['positive']['f1-score'])

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
lo

  0%|          | 0/2094 [00:00<?, ?it/s]

{'negative': {'precision': 0.9711255156157925, 'recall': 0.8696569920844327, 'f1-score': 0.9175946547884186, 'support': 1895}, 'positive': {'precision': 0.3778337531486146, 'recall': 0.7537688442211056, 'f1-score': 0.5033557046979866, 'support': 199}, 'accuracy': 0.8586437440305635, 'macro avg': {'precision': 0.6744796343822036, 'recall': 0.8117129181527691, 'f1-score': 0.7104751797432026, 'support': 2094}, 'weighted avg': {'precision': 0.9147429651234484, 'recall': 0.8586437440305635, 'f1-score': 0.878228107000455, 'support': 2094}}
0.5033557046979866


# Evaluate Submission Test Set

In [ ]:
# Load submission test set
# Change 4 to 0 in load_task1 function to load with all rows
dpm_test = DontPatronizeMe('/content/drive/MyDrive/Colab Notebooks/NLP CW/task4_test.tsv', None) 
dpm_test.load_task1()

In [ ]:
# Get dataframe
sub_test_df = dpm_test.train_task1_df

In [ ]:
sub_test_df.tail()

,par_id,art_id,keyword,country,text,label,orig_label
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...,1,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...,1,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina...",1,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...,1,The United Kingdom is n't going to devolve int...
3831,t_3897,@@26170085,poor-families,ng,This moral battle informed the recent defectio...,1,This moral battle informed the recent defectio...


In [ ]:
print(len(sub_test_df))

3832


In [ ]:
# Remove all other columns besides text, label
sub_test_df = sub_test_df.filter(['text', 'label'])

In [ ]:
# Rename columns to texts, labels to pass to DpmDataset
sub_test_df = sub_test_df.rename(columns={'text': 'texts', 'label': 'labels'})

In [ ]:
print(len(sub_test_df))

3832


In [ ]:
# Load model, tokenizer, format data
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

model_name = './models/dpm_bert_finetuned/' 
model = BERT_dpm.from_pretrained(model_name)


sub_test_dataset = DpmDataset(tokenizer, sub_test_df)
sub_test_loader = DataLoader(sub_test_dataset)

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
lo

In [ ]:
# Find predictions
def predict_test(model, tokenizer, data_loader):

  preds = []

  with torch.no_grad():
    for data in tqdm(data_loader): 

      paras = data['text']
      pred = predict_dpm(paras, tokenizer, model)
      preds.append(pred['prediction'].tolist())

  return preds

preds_task1 = predict_test(model, tokenizer, sub_test_loader)

  0%|          | 0/3832 [00:00<?, ?it/s]

In [ ]:
# Function to save predictions to a file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
# Save predictions 
labels2file([k for k in preds_task1], '/content/drive/MyDrive/Colab Notebooks/NLP CW/task1_2.txt')

# Hyperparameter Tuning

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
val_loader = DataLoader(val_dataset)

In [ ]:

def training_function(config):
    # Hyperparameters
    learning_rate = config["learning_rate"]
    per_device_train_batch_size = config["per_device_train_batch_size"]
    num_train_epochs = config["per_device_train_batch_size"]
    #weight = config["weight"]

    #for step in range(10):

    # Iterative training function
    main_dpm(learning_rate, per_device_train_batch_size, num_train_epochs)

    # Evaluate on positive F1 score
    model_name = './models/dpm_bert_finetuned/' 
    model = BERT_dpm.from_pretrained(model_name)
    report = evaluate(model, tokenizer, val_loader)
    intermediate_score = report['positive']['f1-score']

    # Feed the score back to Tune
    tune.report(mean_loss=intermediate_score)

analysis = tune.run(
    training_function,
    config={
        "learning_rate": tune.grid_search([0.00001, 0.0001]),
        "per_device_train_batch_size": tune.grid_search([8, 16, 32]),
        "num_train_epochs": tune.grid_search([3, 6, 10]),
        #"weight": tune.grid.search{[.5, .5], [.3, .7], [.1, .9]}
    },
    resources_per_trial= {"gpu": 1}
    )

print("Best config: ", analysis.get_best_config(
    metric="F1", mode="max"))